In [330]:
# chamando o arquivo
import pandas as pd

df = pd.read_excel("processadores.xlsx")


In [331]:
# transformar string em float

# transformar os valores de strings em numeros

df['preço'] = df['preço'].str.replace('R$', '') 
df['preço'] = df['preço'].str.replace('.', '',)
df['preço'] = df['preço'].str.replace(',', '.') 
df['preço'] = df['preço'].str.strip() 
df['preço'] = df['preço'].astype(float)

print(df['preço'].head(10))

                        




0    1749.99
1     589.00
2     999.99
3     619.99
4     749.99
5    3399.99
6    1399.99
7    1299.99
8    3099.99
9     999.99
Name: preço, dtype: float64


In [332]:
# biblioteca 

# dicionario com palavras
dicionario_palavras = {
    'Processadores': ['processador', 'processadores'],
    'Intel': ['intel', 'Intel'],
    'Ryzen': ['Ryzen', 'ryzen'],
    'AMD': ['AMD', 'AMd', 'Amd', "amd"]
    
}


In [333]:
# verifica se a palavra esta no dicionario

def contem_palavra_dicionario(texto, dicionario):
    texto = str(texto).lower()
    for palavras in dicionario.values():
        for palavra in palavras:
            if palavra.lower() in texto:
                return True
    return False

In [334]:
# armazena as palavras 

indices_para_remover = []

for i in df.index:
    texto = df.at[i, 'marca']
    if not contem_palavra_dicionario(texto, dicionario_palavras):
        indices_para_remover.append(i)

In [335]:
# apaga da tabela

df.drop(index=indices_para_remover, inplace=True)
df.reset_index(drop=True, inplace=True)

print(df['marca'].head(200))

0      Processador AMD Ryzen 7 5700X3D, 3.0 GHz, (4.1...
1      Processador AMD Ryzen 5 5500, 3.6GHz, Cache 16...
2      Processador AMD Ryzen 5 5600GT, 3.6 GHz, (4.6G...
3      Processador AMD Ryzen 5 5500, 3.6GHz (4.2GHz M...
4      Processador Intel Core i5-12400F, 2.5GHz (4.4G...
                             ...                        
195    Processador AMD Ryzen 9 7900 4.00ghz, 12-core,...
196    Processador Intel I7-2600, 3.80ghz, 8mb, Cache...
197    Processador Intel I7-8700 4.6GHz, Cache 12MB, ...
198    Processador Intel Core I7 10700K 10ª Geração, ...
199    Processador AMD Ryzen Threadripper Pro 5975WX,...
Name: marca, Length: 200, dtype: object


In [336]:
def identificar_fabricante(texto):
    texto = str(texto).lower()
    if any(palavra.lower() in texto for palavra in dicionario_palavras['Intel']):
        return 'Intel'
    elif any(palavra.lower() in texto for palavra in dicionario_palavras['AMD']) or any(palavra.lower() in texto for palavra in dicionario_palavras['Ryzen']):
        return 'AMD'
    else:
        return 'Desconhecido'

In [337]:
df['fabricante'] = df['marca'].apply(identificar_fabricante)

print(df[['marca', 'fabricante']].head(200))

                                                 marca fabricante
0    Processador AMD Ryzen 7 5700X3D, 3.0 GHz, (4.1...        AMD
1    Processador AMD Ryzen 5 5500, 3.6GHz, Cache 16...        AMD
2    Processador AMD Ryzen 5 5600GT, 3.6 GHz, (4.6G...        AMD
3    Processador AMD Ryzen 5 5500, 3.6GHz (4.2GHz M...        AMD
4    Processador Intel Core i5-12400F, 2.5GHz (4.4G...      Intel
..                                                 ...        ...
195  Processador AMD Ryzen 9 7900 4.00ghz, 12-core,...        AMD
196  Processador Intel I7-2600, 3.80ghz, 8mb, Cache...      Intel
197  Processador Intel I7-8700 4.6GHz, Cache 12MB, ...      Intel
198  Processador Intel Core I7 10700K 10ª Geração, ...      Intel
199  Processador AMD Ryzen Threadripper Pro 5975WX,...        AMD

[200 rows x 2 columns]


In [338]:
import re

def extrair_modelo_completo(texto):
    texto = str(texto).strip()

    # Regex para detectar Ryzen, Core, Pentium, Celeron etc.
    match = re.search(
        r'\b(Ryzen\s?[3-9]|Core\s+Ultra\s?[5-9]|Core\s?i[3-9]|Intel\s?i[3-9]|i[3-9]|Pentium(?:\s+Gold|\s+Silver)?|Celeron)\s*-?\s*([A-Za-z]?\d{3,5})([A-Za-z0-9]*)',
        texto,
        re.IGNORECASE
    )

    if match:
        serie = match.group(1).title().strip()
        numero = match.group(2)
        sufixo = match.group(3).upper()
        return f"{serie} {numero}{sufixo}" if sufixo else f"{serie} {numero}"
    
    return None

def extrair_soquete(texto):
    texto = str(texto).strip()

    # Regex para pegar soquetes tipo AM5, AM4, LGA, FCLGA etc.
    match_soquete = re.search(r'\b(AM5|AM4|LGA\s?\d+|FCLGA\s?\d+)\b', texto, re.IGNORECASE)

    if match_soquete:
        return match_soquete.group(1).upper().replace(" ", "")
    
    return None

# Aplicar extrações
df['Modelo'] = df['marca'].apply(extrair_modelo_completo)
df['Soquetes'] = df['marca'].apply(extrair_soquete)

# Remover linhas onde modelo OU soquete não foi encontrado
df = df.dropna(subset=['Modelo', 'Soquetes']).reset_index(drop=True)


In [339]:
df.to_excel("processadores_limpo.xlsx", index=False)

